<a href="https://colab.research.google.com/github/kv3n/shape-analysis/blob/master/shape_preserving_transformations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-graphics
!pip install openmesh
!pip install -U -q PyDrive

In [0]:
from tensorflow_graphics.geometry.transformation import quaternion
from tensorflow_graphics.notebooks import mesh_viewer
import openmesh as om
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

# Loading the Model

I experimented a few ways to do this in Google Colab, but I had to settle for a solution that involves a little bit of work. We are going to have to upload a model to a specific location in Google Drive for this to work.

**Steps**:
- Create a folder called resources in your Google Drive.
- Upload a 3D model file (I downloaded mine from [Google Poly by "Keen Chan"](https://poly.google.com/view/caZFn1bu4Zc)). We only care about the .obj file for now.
- Rename the 3D model file to 'model.obj'
- Authorize google drive access when running the cell below

.. and with that you should be ready to load and view the model.

  
Feel free to alter this block however you feel fit to load the openmesh model!



In [0]:
from google.colab import drive
drive.mount("/content/drive")

# Create

mesh = om.read_trimesh('/content/drive/My Drive/resources/model.obj')
print("Mesh Loaded")

# Build Tensorflow Ready Mesh Data
Tensorflow expects a mesh to be definied as a JSON dicitonary with the following format: 
```
  { 
    'vertices': <array of 3D positions>,
     'faces': <array of 3 indices of vertices that form a traingle>,
      'vertex_colors': <array of colors for each vertex>,
      'material': <a material applied to the model>
  }
```

The vertex_colors and materials are optional. So we will ignore them for now and keep our demo simple. 

I use OpenMesh here to load our 3D model, hence the following cell of code simply just converts an open mesh loaded 3D object into tensorflow 3D mesh data. 

*Perhaps there is an easier tensorflow way of doing this that I haven't come across yet!?


In [0]:
# Load Vertices
vertices = []
for point in mesh.points():
  vertices.append(point)
vertices = np.array(vertices, dtype=np.float32)

# Load Triangles
faces = []
for fh in mesh.faces():
  face = []
  for vh in mesh.fv(fh):
    face.append(vh.idx())
  faces.append(face)
faces = np.array(faces)

In [0]:
def build_mesh_data(mesh_vertices):
  mesh_data = {'vertices': mesh_vertices[:, ...], 'faces': faces[:, ...]}
  return mesh_data

mesh_viewer.Viewer(build_mesh_data(vertices))

# Applying Shape Preserving Operations
We will apply our simple shape preserving operations one by one on the original mesh. That is, these operations are applied independently. The result of one does not affect the next operation.

Notice while apply our operations, we are altering only our vertices. That is, the operation applies straight to the vertices and the faces don't change. That said, I am not a fan of how I have to keep reconstructing the dictionary.

Note that the camera is located at the same place throughout these changes. It is only the vertices that change positions.

## Translation
Here we try to translate the object by 1 unit in the x-axis (that is, to the right). The math is simple, <br>
```
new_position = current_position + distance * direction
```

In the matrix world, the translation matrix would look like
```
[1, 0, 0, x
 0, 1, 0, y
 0, 0, 1, z
 0, 0, 0, 1]
```

In [0]:
translation = 1.0 * np.array([1.0, 0.0, 0.0], dtype=np.float32)
translated_vertices = vertices + translation

mesh_viewer.Viewer(build_mesh_data(translated_vertices))

## Rotation

Rotation in computer graphics is done using Quaternions. Quaternions is a deep subject of its own and I highly recommend watching [this video](https://www.youtube.com/watch?v=zjMuIxRvygQ). It is a good quick intro to quaternions, but if interested you should check out [Visualizing Quaternions by Andrew J Hanson.](https://books.google.com/books/about/Visualizing_Quaternions.html?id=CoUB09xzme4C&printsec=frontcover&source=kp_read_button#v=onepage&q&f=false)

In our example we try to rotate the model by 90 degrees (1.57 radians) counter clock-wise about the y-axis making the rover face us.

The rotation matrix about the z axis would look like

```
[cos theta, -sin theta, 0, 0
 sin theta, cos theta , 0, 0
    0    ,     0      , 1, 0
    0    ,     0      , 0, 1]
```

In [0]:
rotation = quaternion.from_euler(np.array([0.0, 1.57, 0.0], dtype=np.float32))
rotated_vertices = quaternion.rotate(vertices, rotation)

mesh_viewer.Viewer(build_mesh_data(rotated_vertices))

## Scaling

We will perform what is called Uniform Scaling. That is, we will scale equally along the x-axis, y-axis and z-axis. Scaling matrix is generally represented as a diagonal matrix. That is, a matrix with 0s everywhere but the diagonal.

In [0]:
scale = 1.5 * np.eye(3)  # np.eye produces an identity matrix
scaled_vertices = np.transpose(np.matmul(scale, np.transpose(vertices)))  
# Notice the double transpose. This is actually not needed.
# We could have done np.matmul(vertices, scale).
# But this does not play well with me because traditionally transformations ..
# .. are to be pre-multiplied with the points. But because our vertices have ..
# .. (x, 3) shape, I had to transpose it to get (3, x) shape for matrix ..
# .. multiplication compatibility.
mesh_viewer.Viewer(build_mesh_data(scaled_vertices))

# Order Absolutely Matters

With the following code I will demonstrate why order of operations matter when it comes to combining transformations.


1.   First we will apply a rotation and then a translation. In matrix multiplication world this would look like
```
new_position = translation_matrix * rotation_matrix * current_position
```
2.   Then we will apply a translation followed by a rotation.
```
new_position = rotation_matrix * translation_matrix * current_position
```

In [0]:
order_1 = quaternion.rotate(vertices, rotation) + translation
order_2 = quaternion.rotate(vertices + translation, rotation)
mesh_viewer.Viewer(build_mesh_data(order_1))
mesh_viewer.Viewer(build_mesh_data(order_2))

The easiest rule to remember is the **SRT rule.** You always apply scaling first, rotation next and finally translation. In matrix multiplication world the order is reversed. You would end up doing

```
new_position = translation * rotation * scaling * current_position
```